# **Tabela para População**

  Dados do Censo e Estimativa da população do IBGE. Utiliando dados da Tabela 6579- População residente estimada, e variável 9324- População residente estimada. Para o ano de 2007 foram agregados os dados disponíveis no repositório do IBGE.

  Esta tabela foi incluída pelo fato de ser uma variável utilizada no cáluclo do índice de partição das receitas do FPM, ICMS, IPI, FEP e ICMS Desoneração.
  
  Periodocidade: Anual

  Fonte dos dados:
   * https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html
   * https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-bq

## Objetivos:
 * Baixar os dados da estimativa da população dos municípios estudados;
 * Ajustar variável para o ano inicial da consulta que é 2001;
 * Consumir API do IBGE;
 * Fazer o consumo da API por etapas para evitar problemas com limitações da quantidade de regristros, pois cada requisição permite retornar no máximo 100.000 valores.
 * Unificar com os dados do Censo do IBGE;
 * Criar um único arquivo somente com os municipios estudados:


| Cidadade | UF | Código IBGE |
|--|--|---|
| Belém | PA | 1501402 |
| Belo Horizonte | MG | 3106200 |
| Campo Grande | MS | 5002704 |
| Fortaleza | CE | 2304400|
| Maceió | AL | 2704302 |
| Palmas | TO | 1721000 |
| Salvador | BA | 2927408 |
| São Luís | MA | 2111300 |

Informações para consumo da API do IBGE disponíveis no site https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-bq

Acessado em 10 de setembro de 2022.

Bibliotecas para ler os arquivos

 * requests para conectar e recuperar conteúdo da API;
 * datetime manipular dados de data;
 * os para a navegação e manipulação dos arquivos no sistema operacional;


## Incluir dados da Estimativa da população

    Os dados de estimativa da população são utilizados pelo Tribunal de Contas da União (TCU) para calcular o índice de partição das receitas do FPM.

In [ ]:
# -*- encoding: utf-8 -*-
import requests
import pandas as pd
import json
import os

from pandas import json_normalize
from datetime import datetime


In [ ]:
print(datetime.now())
# Lista para guardas os dados
dataset_list = []
dtset_lst = []
# Aqui são as variáveis que temos que pegar
list_cabe = ["idMuni", "Municipio", "Ano", "Populacao", "UF", "PopUF"]

2022-11-19 18:28:40.640284


In [ ]:
# @title Variáveis
# Cria variável para caminho de armazenamento dos arquivos
Caminho = "E://MESTRADO\DISSERTACAO/Dados_Abertos/"
basedir = Caminho + "Estimativa_Populacao/API_IBGE/DADOS/"

# Nome do arquivo
nomearq = basedir + "EstimativaPopulacao.csv"
#print(nomearq)

# Define Datas em formato texto
AnoAtual = datetime.now().year - 1

# Separa o Ano Atual e subtrai 1 anos, pois o TCU e demais Órgão de controle
# utilizam a população de 2 anos anteriores para calcular os índices de distribuição de recursos
maiordata = str(AnoAtual) + "-01-01"
print(maiordata)
# Esta data deve ficar fixa pois é a menor que a consulta disponibiliza
menordata = '2001-01-01'

# Guarda a linha que será gravada no arquivo
tmp = ""

# Armazena os anos da consulta
apiPeriodo = ""

# Lista de Cidades para consulta precisa de todas as capitais, pois há necessidade
# de criar os campos de percentual entre capitais
listCid = []


2021-01-01


In [ ]:
# @title Lista de Capitais do IBGE do Censo de 2010
urlcid = "https://servicodados.ibge.gov.br/api/v3/agregados/1287/periodos/2010/variaveis/591?localidades=N6"
resp = requests.get(urlcid)
cidjson = resp.json()
cidjson

[{'id': '591',
  'variavel': 'População nos municípios das capitais',
  'unidade': 'Pessoas',
  'resultados': [{'classificacoes': [],
    'series': [{'localidade': {'id': '1100205',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Porto Velho - RO'},
      'serie': {'2010': '428527'}},
     {'localidade': {'id': '1200401',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Rio Branco - AC'},
      'serie': {'2010': '336038'}},
     {'localidade': {'id': '1302603',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Manaus - AM'},
      'serie': {'2010': '1802014'}},
     {'localidade': {'id': '1400100',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Boa Vista - RR'},
      'serie': {'2010': '284313'}},
     {'localidade': {'id': '1501402',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Belém - PA'},
      'serie': {'2010': '1393399'}},
     {'localidade': {'id': '1600303',
       'nivel': {'id': 'N

In [ ]:
objcid = dict()
objcid = list(dict(cidjson[0]['resultados'][0]).get("series"))
print(objcid)

# Verifica os valores do retorno das series
lt = 0
while lt < len(objcid):
  objloc = dict(objcid[lt])
  print(lt)
  print(objloc)
  listCid.append(dict(objloc.get("localidade")).get('id'))
  # 'localidade': {'id': '2111300',
  print(dict(objloc.get("localidade")).get('id'))
  print(dict(objloc.get("localidade")).get("nome"))
  print("-------------------")
  # Preenche a lista de cidades
  listCid

  lt = lt + 1

[{'localidade': {'id': '1100205', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Porto Velho - RO'}, 'serie': {'2010': '428527'}}, {'localidade': {'id': '1200401', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Rio Branco - AC'}, 'serie': {'2010': '336038'}}, {'localidade': {'id': '1302603', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Manaus - AM'}, 'serie': {'2010': '1802014'}}, {'localidade': {'id': '1400100', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Boa Vista - RR'}, 'serie': {'2010': '284313'}}, {'localidade': {'id': '1501402', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Belém - PA'}, 'serie': {'2010': '1393399'}}, {'localidade': {'id': '1600303', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Macapá - AP'}, 'serie': {'2010': '398204'}}, {'localidade': {'id': '1721000', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Palmas - TO'}, 'serie': {'2010': '228332'}}, {'localidade': {'id': '2111300', 'nivel': {'id': 'N6', 'nome': 'Munic

In [ ]:
# Lista de Cidades para consulta precisa de todas as capitais, pois há necessidade
# de criar os campos de percentual entre capitais
print(listCid)

['1100205', '1200401', '1302603', '1400100', '1501402', '1600303', '1721000', '2111300', '2211001', '2304400', '2408102', '2507507', '2611606', '2704302', '2800308', '2927408', '3106200', '3205309', '3304557', '3550308', '4106902', '4205407', '4314902', '5002704', '5103403', '5208707', '5300108']


Os símbolos do strftime são:

 * %d - O dia do mês no formato de um número decimal (de 01 a 31);
 * %m - O mês no formato de um número decimal (de 01 a 12);
 * %Y - O ano no formato de um número decimal de 4 dígitos.

 Disponíveis em: https://man7.org/linux/man-pages/man3/strftime.3.html

In [ ]:
# cria um diretório para armazenar o arquivo comparâmetro para não criar se já existir
os.makedirs(os.path.abspath(os.path.dirname(basedir)), exist_ok=True)

# Transforma em diretório
basedir = os.path.abspath(os.path.dirname(basedir))

In [ ]:
# @title Apaga o arquivo se foi criado
if os.path.isfile(os.path.join(basedir, nomearq)):
  # Se existe apaga o arquivo
  os.remove(os.path.join(basedir, nomearq))


In [ ]:
# Cria o intervalo de datas
datesStart = pd.date_range(start=menordata, end=maiordata, freq = 'YS')
# As frequencia podem ser consultadas na documentação disponível em:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

print("Intervalo Incial:", datesStart)

Intervalo Incial: DatetimeIndex(['2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01',
               '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01',
               '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01',
               '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01',
               '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01',
               '2021-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')


In [ ]:
# Seleciona apenas os anos e transforma em texto
apiPeriodo = '|'.join([datetime.strftime(dt, format="%Y") for dt in datesStart])

# Variável com os períodos de consulta, no caso os anos
print(apiPeriodo)

2001|2002|2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021


In [ ]:
#https://servicodados.ibge.gov.br/api/v3/
# agregados/6579- População residente estimada,
# /periodos/2001|2002|2003|2004|2005|2006|2008|2009|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021
# /variaveis/9324- População residente estimada
#?localidades=N3[21]  Para consultar por UF
#?localidades=N6[2111300]  Para consultar por Município

# Consulta Estimativa da população
#https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2001|2002|2003|2004|2005|2006|2008|2009|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021/variaveis/9324?localidades=?localidades=N3[21]|N6[2111300]

# Consulta Contagem 2007
#"https://servicodados.ibge.gov.br/api/v3/agregados/793/periodos/2007/variaveis/93?localidades=N3[21]|N6[2111300]"

# Consulta do Censo
#https://servicodados.ibge.gov.br/api/v3/agregados/1309/periodos/2000|2010/variaveis/93?localidades=N3[21]|N6[2111300]&classificacao=2[0]|11277[0]

# variaveis/9324??localidades=N3[21]|N6[2111300]

def cria_url(uano, idMunic, idUF, utabela, uvariavel):
  # String de pesquisa da url
  s = ""
  s = "agregados/" + utabela
  s = s + "/periodos/" + uano
  s = s + "/variaveis/" + uvariavel
  s = s + "?localidades=N3[" + idUF
  s = s + "]|N6[" + idMunic
  s = s + "]"

  # Monta base da consulta
  url = "https://servicodados.ibge.gov.br/api/v3/" + s

  print("Município: ", idMunic)
  print('------------------------------------------------------------------------------------')
  print("webdriver gets url....")
  print(url)

  return url

In [ ]:
# Função para ler os arquivos das urls
def Ler_Arquivo_URL(lcodmuni, luresp):
  # Garda na variável a resposta para o método GET do site
  resposta = requests.get(luresp)
  arq_json = resposta.json()

  # Transfora em dicionário
  objeto = dict()
  objeto = list(dict(arq_json[0]['resultados'][0]).get("series"))
  print(objeto)

  # Verifica os valores do retorno das series
  lt = 0
  while lt < len(objeto):
    objloc = dict(objeto[lt])
    print(objloc)
    # 'localidade': {'nome': 'São Luís (MA)'
    print(dict(objloc.get("localidade")).get('nome'))
    # 'localidade': {'id': '2111300',
    print(dict(objloc.get("localidade")).get('id'))
    print(dict(dict(objloc.get("localidade")).get("nivel")).get('id'))

    Ler_Lista_Campos(dict(objloc.get("serie")),
                    lcodmuni,
                    dict(objloc.get("localidade")).get('nome'),
                    dict(dict(objloc.get("localidade")).get("nivel")).get('id')
                    )
    lt = lt + 1

  print("--------------------------------------------")

In [ ]:
def Ler_Lista_Campos(lcolecao, lcodigo, lnome, lnivel):
  # Varrendo a collection
  for key, value in lcolecao.items():
    # 'serie': {'2001': '889129',
    tmp = ""
    list_campos = []
    list_campos.append(lcodigo)
    list_campos.append(lnome)
    list_campos.append(key)
    list_campos.append(value)
    # Cria o campo de Indice
    list_campos.append(str(lcodigo) + str(key))
    # Cria a lista para o Dataset
    if lnivel in "N3":
      dtset_lst.append(list_campos)
    else:
      dataset_list.append(list_campos)

In [ ]:
#[                                               # Inicia com uma lista #
#  {'id': '9324',                                #  um dicionário #
#   'variavel': 'População residente estimada',
#   'unidade': 'Pessoas',
#   'resultados': [                               # Nova lista #
#                  {'classificacoes': [],         #  um dicionário #
#                   'series': [                   # Mais uma lista #
#                              {'localidade': {'id': '2111300',
#                                               'nivel': {
#                                                         'id': 'N6',
#                                                         'nome': 'Município'
#                                                         },
#                                               'nome': 'São Luís (MA)'
#                                               },
#                               'serie': {'2001': '889129',
#                                         '2002': '906567',
#                                         '2003': '923526',
#                                         '2004': '959124',
#                                         '2005': '978824',
#                                         '2006': '998385',
#   FALTA 2007
#                                         '2008': '986825',
#                                         '2009': '997098',
#   NÃO HÁ ESTIMATIVA PARA 2010 - CENSO
#                                         '2011': '1027430',
#                                         '2012': '1039610',
#                                         '2013': '1053922',
#                                         '2014': '1064197',
#                                         '2015': '1073893',
#                                         '2016': '1082935',
#                                         '2017': '1091868',
#                                         '2018': '1094667',
#                                         '2019': '1101884',
#                                         '2020': '1108975',
#                                         '2021': '1115932'
#                                        }
#                             }]
#                 }]
#  }
#]

**Observações:**

 *  Os dados de 2007 não estão disponíveis na mesma APi do IBGE, mas na de contagem da população consta na tabela 793- População residente, variável 93 - População residente;
 * O censo é feito a cada 10 anos e seus dados ficam na Tabela 1309- População residente, por sexo, situação e localização da área, assim foi utilizada a variável 93- População residente.

In [ ]:
# @title Laço para buscar os arquivos
i = 0
while i < len(listCid):

  print(str(listCid[i])[:2])
  #cria string da url da API da Estimativa
  url = cria_url(apiPeriodo, listCid[i], str(listCid[i])[:2], "6579", "9324")

  # Consome a API lendo o arquivo json criado
  Ler_Arquivo_URL(listCid[i], url)

  # Cria string da url da API da contagem de 2007
  url = cria_url("2007", listCid[i], str(listCid[i])[:2], "793", "93")

  # Consme essa API de 2007
  Ler_Arquivo_URL(listCid[i], url)

 # Cria string da url da API dos Censos de 2000 e 2010
  resposta = cria_url("2000|2010", listCid[i], str(listCid[i])[:2], "1309", "93")

  # Consme essa API dos Censos
  Ler_Arquivo_URL(listCid[i], resposta)

  #print(dataset_list)
  # Incrementa a contagem das datas
  i = i + 1


11
Município:  1100205
------------------------------------------------------------------------------------
webdriver gets url....
https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2001|2002|2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021/variaveis/9324?localidades=N3[11]|N6[1100205]
[{'localidade': {'id': '11', 'nivel': {'id': 'N3', 'nome': 'Unidade da Federação'}, 'nome': 'Rondônia'}, 'serie': {'2001': '1407886', '2002': '1431777', '2003': '1455907', '2004': '1562085', '2005': '1534594', '2006': '1562417', '2008': '1493565', '2009': '1503928', '2011': '1576455', '2012': '1590011', '2013': '1728214', '2014': '1748531', '2015': '1768204', '2016': '1787279', '2017': '1805788', '2018': '1757589', '2019': '1777225', '2020': '1796460', '2021': '1815278'}}, {'localidade': {'id': '1100205', 'nivel': {'id': 'N6', 'nome': 'Município'}, 'nome': 'Porto Velho (RO)'}, 'serie': {'2001': '342264', '2002': '347844', '2003': '353961', '2004'

In [ ]:
# @title Cria os datasets e mescla-os
datapop = pd.merge(pd.DataFrame(dataset_list, columns=["idMuni", "Municipio", "Ano", "Populacao", "Indice"]),
                   pd.DataFrame(dtset_lst, columns=["idMuni", "UF", "Ano", "PopUF", "Indice"]),
                   how = 'outer')
datapop['Populacao'] = pd.to_numeric(datapop['Populacao'], errors='coerce')

In [ ]:
# Fazendo um Group by no ano para ter o valor da população das capitais
datapop = pd.merge(pd.DataFrame(datapop.groupby('Ano')['Populacao'].sum(), columns=["Populacao"]).reset_index().rename(columns={'Populacao': 'PopCapitais'}),
                   datapop,
                   how='outer')

In [ ]:
datapop

,Ano,PopCapitais,idMuni,Municipio,Populacao,Indice,UF,PopUF
0,2000,40388490,1100205,Porto Velho (RO),334585,11002052000,Rondônia,1377792
1,2000,40388490,1200401,Rio Branco (AC),252885,12004012000,Acre,557226
2,2000,40388490,1302603,Manaus (AM),1403796,13026032000,Amazonas,2813085
3,2000,40388490,1400100,Boa Vista (RR),200383,14001002000,Roraima,324152
4,2000,40388490,1501402,Belém (PA),1279861,15014022000,Pará,6189550
...,...,...,...,...,...,...,...,...
589,2021,50916038,4314902,Porto Alegre (RS),1492530,43149022021,Rio Grande do Sul,11466630
590,2021,50916038,5002704,Campo Grande (MS),916001,50027042021,Mato Grosso do Sul,2839188
591,2021,50916038,5103403,Cuiabá (MT),623614,51034032021,Mato Grosso,3567234
592,2021,50916038,5208707,Goiânia (GO),1555626,52087072021,Goiás,7206589


In [ ]:
# @title Salva o arquivo
datapop.to_csv(nomearq, index=False, sep=';')

In [ ]:
dtset_lst

[['1100205', 'Rondônia', '2001', '1407886', '11002052001'],
 ['1100205', 'Rondônia', '2002', '1431777', '11002052002'],
 ['1100205', 'Rondônia', '2003', '1455907', '11002052003'],
 ['1100205', 'Rondônia', '2004', '1562085', '11002052004'],
 ['1100205', 'Rondônia', '2005', '1534594', '11002052005'],
 ['1100205', 'Rondônia', '2006', '1562417', '11002052006'],
 ['1100205', 'Rondônia', '2008', '1493565', '11002052008'],
 ['1100205', 'Rondônia', '2009', '1503928', '11002052009'],
 ['1100205', 'Rondônia', '2011', '1576455', '11002052011'],
 ['1100205', 'Rondônia', '2012', '1590011', '11002052012'],
 ['1100205', 'Rondônia', '2013', '1728214', '11002052013'],
 ['1100205', 'Rondônia', '2014', '1748531', '11002052014'],
 ['1100205', 'Rondônia', '2015', '1768204', '11002052015'],
 ['1100205', 'Rondônia', '2016', '1787279', '11002052016'],
 ['1100205', 'Rondônia', '2017', '1805788', '11002052017'],
 ['1100205', 'Rondônia', '2018', '1757589', '11002052018'],
 ['1100205', 'Rondônia', '2019', '177722

In [ ]:
#_json

In [ ]:
#_json[0]

In [ ]:
#objloc

In [ ]:
#info = json.loads(objloc.get("serie"))


In [ ]:
#objloc.get("serie")

In [ ]:
#df = pd.read_json(objloc.get("serie"), orient ='index')
#print(df)

In [ ]:
#df = json_normalize(info['serie']) #Results contain the required data
#print(df)


In [ ]:
print(datetime.now())
# Tempo aproximado de 1 minutos

2022-11-18 14:10:06.808583
